In [15]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("MastodonBatchProcessing") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.25") \
    .getOrCreate()

# Paramètres de connexion PostgreSQL
db_url = "jdbc:postgresql://projetfinal5spar-postgres-1:5432/mastodon_data"
db_properties = {
    "user": "user",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

# Charger les données historiques de PostgreSQL
df = spark.read \
    .jdbc(url=db_url, table="mastodon", properties=db_properties)

# Afficher le schéma des données
df.printSchema()

# Définir la valeur de X pour filtrer les utilisateurs actifs
X = 2 # Par exemple, utilisateurs ayant plus de 10 pouets

# Étape 1: Filtrer le  nombre de toots par utilisateurs 
X_toots_users_df = df.groupBy("user_id").count().filter(F.col("count") >= X)

# Étape 2: Afficher les utilisateurs ayant plus de X toots
print("Utilisateurs ayant plus de {} pouets :".format(X))
X_toots_users_df.show()

# Étape 3: Joindre pour obtenir les détails des toots de ces utilisateurs
details_df = df.join(X_toots_users_df.select("user_id"), on="user_id", how="inner")

# Afficher les toots des utilisateurs actifs
print("Pouets des utilisateurs actifs :")
details_df.show()

# Regrouper par heure et hashtags
tweets_by_hour_df = df \
    .withColumn("hour", F.date_trunc("hour", "timestamp")) \
    .groupBy("hour", "hashtags") \
    .agg(
        F.count("*").alias("total_tweets"),
        F.first("content").alias("first_tweet")  # Juste un exemple d'agrégation
    )

# Identifier le hashtag le plus fréquent
most_frequent_hashtags_df = tweets_by_hour_df \
    .groupBy("hashtags") \
    .agg(F.sum("total_tweets").alias("total_tweets")) \
    .orderBy(F.desc("total_tweets")) \
    .limit(1)

# Compter le nombre total de toots par jour
daily_tweet_count_df = df \
    .withColumn("day", F.to_date("timestamp")) \
    .groupBy("day") \
    .agg(F.count("*").alias("total_tweets"))


# Afficher les résultats
print("Tweets par heure et hashtags:")
tweets_by_hour_df.show()

print("Hashtag le plus fréquent:")
most_frequent_hashtags_df.show()

print("Nombre total de pouets par jour:")
daily_tweet_count_df.show()



# Calculer la longueur moyenne des pouets par type d'optimisation
print(f"Calcul de la longueur moyenne des pouets par type d'optimisation")
# Optimisation : repartition, coalesce, caching

# Mesurer le temps d'exécution avant optimisation
print("Longueur moyenne des pouets par utilisateur sans optimisation")
start_time = time.time()

average_length_df = df.withColumn("content_length", F.length("content")) \
    .groupBy("user_id") \
    .agg(F.avg("content_length").alias("avg_length"))
average_length_df.show()

end_time = time.time()
print(f"Temps d'exécution de la Longueur moyenne des pouets par utilisateur avant optimisation : {end_time - start_time} secondes")

# Appliquer la repartition
print("Optimisation par Repartition")
repartition_df = df.repartition(10)

start_time = time.time()

average_length_df = repartition_df.withColumn("content_length", F.length("content")) \
    .groupBy("user_id") \
    .agg(F.avg("content_length").alias("avg_length"))
average_length_df.show()

end_time = time.time()
print(f"Temps d'exécution de la Longueur moyenne des pouets par utilisateur par repartition : {end_time - start_time} secondes")


# Appliquer le coalesce
print("Optimisation par Coalesce")
coalesce_df = df.coalesce(10)

start_time = time.time()

average_length_df = coalesce_df.withColumn("content_length", F.length("content")) \
    .groupBy("user_id") \
    .agg(F.avg("content_length").alias("avg_length"))
average_length_df.show()

end_time = time.time()
print(f"Temps d'exécution de la Longueur moyenne des pouets par utilisateur par coalesce : {end_time - start_time} secondes")

# Appliquer le cache
print("Optimisation par Caching")
df.cache

start_time = time.time()

average_length_df = df.withColumn("content_length", F.length("content")) \
    .groupBy("user_id") \
    .agg(F.avg("content_length").alias("avg_length"))
average_length_df.show()

end_time = time.time()
print(f"Temps d'exécution de la Longueur moyenne des pouets par utilisateur par caching : {end_time - start_time} secondes")

# Arrêter la session Spark
spark.stop()


root
 |-- id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- content: string (nullable = true)
 |-- language: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- favourites_count: integer (nullable = true)
 |-- reblogs_count: integer (nullable = true)

Utilisateurs ayant plus de 2 pouets :
+-------+-----+
|user_id|count|
+-------+-----+
|  67618|    2|
+-------+-----+

Pouets des utilisateurs actifs :
+-------+------------------+-----+---------------+-------------------+--------------------+--------+--------------------+----------------+-------------+
|user_id|                id| user|followers_count|          timestamp|             content|language|            hashtags|favourites_count|reblogs_count|
+-------+------------------+-----+---------------+-------------------+----

SyntaxError: invalid syntax (1364580831.py, line 1)